In [1]:
# activate line execution
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# plotly
import plotly.express as px  # (version 4.7.0 or higher)
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# !pip install wandb -qqq
import wandb
# wandb.login()
api = wandb.Api()
import os

os.environ["WANDB_API_KEY"] = "0f780ac8a470afe6cb7fc474ff3794772c660465"

os.environ["WANDB_NOTEBOOK_NAME"] = "jup_res"


# selected_tag = 'Pred'

runs = api.runs("hokarami/TEDAM4")


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /home/hokarami/.netrc


In [3]:
import pandas as pd 


# Project is specified by <entity/project-name>
def dl_runs(all_runs, selected_tag=None):




    summary_list, config_list, name_list = [], [], []
    for run in runs: 

        if (selected_tag not in run.tags) and (selected_tag is not None):
            continue
        # .summary contains the output keys/values for metrics like accuracy.
        #  We call ._json_dict to omit large files 
        summary_list.append(run.summary._json_dict)

        # .config contains the hyperparameters.
        #  We remove special values that start with _.
        

        config_list.append(
            {k: v for k,v in run.config.items()
            if not k.startswith('_')})

        # .name is the human-readable name of the run.
        name_list.append(run.name)

    runs_df = pd.DataFrame({
        "summary": summary_list,
        "config": config_list,
        "name": name_list
        })

    # runs_df.to_csv("project.csv")
    return runs_df

In [4]:
# q = runs_df['name'].str.contains('\[R2-')
# runs_df = runs_df[q]


runs_df = dl_runs(runs, selected_tag=None)

runs_df
runs_df.iloc[0].summary.keys()
runs_df.iloc[0].config.keys()

,summary,config,name
0,"{'Valid-NextTime/RMSE': 3.5860057019528284, 'V...",{'w': '[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1....,[NN]1375320-
1,"{'Test-CIF/#events': 20731, 'time-Time/train_e...",{'w': '[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1....,[NN]1379752-
2,"{'Obj': 0.004104052267313776, 'Valid-NextType(...","{'w': [1, 1, 1], 'lr': 0.003, 'log': 'log.txt'...",[NN]1377295-
3,"{'Valid-CIF/LL-#events': 0.4165912158109405, '...","{'w': [1, 1, 1], 'lr': 0.003, 'log': 'log.txt'...",[NN]1371943-
4,"{'_wandb': {'runtime': 64}, 'Valid-NextType(ML...","{'w': [1, 1, 1], 'lr': 0.003, 'log': 'log.txt'...",[NN]1371309-
5,"{'_step': 40, 'Train-NextTime/rmse_norm': 699....","{'w': [1, 1, 1], 'lr': 0.003, 'log': 'log.txt'...",[NN]1374428-
6,"{'_timestamp': 1675248026.042684, 'Train-loss/...","{'w': [1, 1, 1], 'lr': 0.003, 'log': 'log.txt'...",[NN]1378829-
7,"{'_timestamp': 1675247936.869605, 'Test-CIF/#e...","{'w': [1, 1, 1], 'lr': 0.003, 'log': 'log.txt'...",[NN]1371208-
8,{'Valid-NextType(ML)/f1-weighted': 0.590584631...,"{'w': [1, 1, 1], 'lr': 0.003, 'log': 'log.txt'...",[NN]1373792-
9,"{'max_obj_val': 0.12019204601354788, 'Test-Nex...","{'w': [1, 1, 1], 'lr': 0.003, 'log': 'log.txt'...",[NN]1379506-


dict_keys(['Valid-NextTime/RMSE', 'Valid-CIF/LL-#events', 'Train-NextType(ML)/auc-ovo-micro', 'Test-NextTime/RMSE', 'Test-NextTime/msae', 'Train-NextTime/RMSE', 'Best-Test-CIF/LL-#events', 'Valid-NextTime/rmse_norm', 'Valid-CIF/NLL', 'Train-loss/pred_label', 'Valid-NextType(ML)/auc-ovo-weighted', 'Train-NextType(ML)/precision-weighted', 'Train-acc', 'Train-CIF/#events', 'Train-loss/pred_next_type', 'Test-NextType(ML)/auc-ovo-weighted', '_step', 'time-Time/train_epoch', 'time-Time/valid_epoch', 'Train-NextType(ML)/auc-ovo-macro', 'Test-NextType(ML)/auc-PR-weighted', '_timestamp', 'Valid-NextTime/msae', 'Test-NextType(ML)/auc-ovo-micro', 'Train-NextType(ML)/auc-ovo-weighted', '_runtime', 'Test-NextTime/rmse_norm', 'Train-NextTime/rmse_norm', 'Valid-NextType(ML)/auc-ovo-micro', 'Test-NextType(ML)/recall-weighted', 'Valid-NextType(ML)/recall-weighted', 'Valid-NextType(ML)/precision-weighted', 'Test-CIF/LL-#events', 'Train-NextType(ML)/recall-weighted', 'Train-NextTime/msae', 'Train-NextTyp

dict_keys(['w', 'lr', 'log', 'mod', 'per', 'cuda', 'data', 'date', 'demo', 'prof', 'INPUT', 'epoch', 'split', 'state', 'w_pos', 'wandb', 'ES_pat', 'OUTPUT', 'device', 'freeze', 'run_id', 'smooth', 'te_d_k', 'te_d_v', 'w_time', 'dataset', 'int_dec', 'setting', 'w_class', 'w_event', 'run_name', 'run_path', 'te_d_rnn', 'time_enc', 'TE_config', 'event_enc', 'next_mark', 'num_demos', 'num_marks', 'num_types', 'pos_alpha', 'te_d_mark', 'te_d_time', 'te_n_head', 'type_loss', 'CIF_config', 'DAM_config', 'batch_size', 'data_label', 'num_states', 'pos_weight', 'str_config', 'te_d_inner', 'te_dropout', 'testloader', 'demo_config', 'mark_detach', 'te_n_layers', 'test_center', 'trainloader', 'user_prefix', 'validloader', 'w_pos_label', 'label_config', 'sample_label', 'weight_decay', 'hparams2write', 'balanced_batch', 'label_loss_fun', 'pred_loss_func', 'w_sample_label', 'next_time_config', 'next_type_config', 'transfer_learning'])

# Table I
Effect of time concatenation and single+mark

In [6]:
df_filt = dl_runs(runs, selected_tag=None)
df_filt


,summary,config,name
0,"{'Valid-NextTime/RMSE': 3.5860057019528284, 'V...",{'w': '[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1....,[NN]1375320-
1,"{'Test-CIF/#events': 20731, 'time-Time/train_e...",{'w': '[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1....,[NN]1379752-
2,"{'Obj': 0.004104052267313776, 'Valid-NextType(...","{'w': [1, 1, 1], 'lr': 0.003, 'log': 'log.txt'...",[NN]1377295-
3,"{'Valid-CIF/LL-#events': 0.4165912158109405, '...","{'w': [1, 1, 1], 'lr': 0.003, 'log': 'log.txt'...",[NN]1371943-
4,"{'_wandb': {'runtime': 64}, 'Valid-NextType(ML...","{'w': [1, 1, 1], 'lr': 0.003, 'log': 'log.txt'...",[NN]1371309-
5,"{'_step': 40, 'Train-NextTime/rmse_norm': 699....","{'w': [1, 1, 1], 'lr': 0.003, 'log': 'log.txt'...",[NN]1374428-
6,"{'_timestamp': 1675248026.042684, 'Train-loss/...","{'w': [1, 1, 1], 'lr': 0.003, 'log': 'log.txt'...",[NN]1378829-
7,"{'_timestamp': 1675247936.869605, 'Test-CIF/#e...","{'w': [1, 1, 1], 'lr': 0.003, 'log': 'log.txt'...",[NN]1371208-
8,{'Valid-NextType(ML)/f1-weighted': 0.590584631...,"{'w': [1, 1, 1], 'lr': 0.003, 'log': 'log.txt'...",[NN]1373792-
9,"{'max_obj_val': 0.12019204601354788, 'Test-Nex...","{'w': [1, 1, 1], 'lr': 0.003, 'log': 'log.txt'...",[NN]1379506-


In [15]:
i_groupby_row =['dataset']
i_groupby_col = ['time_enc']
i_agg = ['split']
i_metric = ['Best-Test-CIF/LL-#events', 'Test-NextType(ML)/auc-ovo-weighted','Test-NextType(ML)/f1-weighted']
# i_metric = ['Test-CIF/NLL-#events']
i_stats=['mean','std']
# i_stats=['mean']

#df_filt.iloc[0].config.keys()

d_name = [x for x in df_filt.name]

df_config = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.config]    )

df_summary = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.summary]    )

df = pd.concat([df_config, df_summary],axis=1)
# df.name = df_filt.name
df = df[i_groupby_row + i_groupby_col + i_agg + i_metric]


# df = df.melt(id_vars=i_groupby_row + i_groupby_col + i_agg ,value_vars=i_metric, var_name='Metric')




a0 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].size().rename('runs')

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(i_stats).round(3)

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(lambda x:str(round(x.mean(),2))  + ' (' + str(round(x.std(),2))  +')'  ).reset_index() #  

# a4 = df.groupby(i_by)[i_metric].apply(lambda x:x.round(3).tolist())   # x.round(3).tolist()
a1
a1.columns
a1.reset_index().pivot(index=i_groupby_row+i_metric,columns=i_groupby_col)



# a1.to_excel("aaa.xlsx")

,dataset,time_enc,split,variable,value
0,synthea_full,sum,0,Best-Test-CIF/LL-#events,0.000000
1,synthea_full,concat,0,Best-Test-CIF/LL-#events,0.000000
2,Retweets,sum,4,Best-Test-CIF/LL-#events,0.000000
3,Retweets,concat,4,Best-Test-CIF/LL-#events,0.482756
4,Retweets,sum,3,Best-Test-CIF/LL-#events,0.228721
...,...,...,...,...,...
73,SO,concat,2,Test-NextType(ML)/f1-weighted,NaN
74,SO,sum,1,Test-NextType(ML)/f1-weighted,NaN
75,SO,concat,1,Test-NextType(ML)/f1-weighted,NaN
76,SO,sum,0,Test-NextType(ML)/f1-weighted,NaN


,dataset,time_enc,Best-Test-CIF/LL-#events,Test-NextType(ML)/auc-ovo-weighted,Test-NextType(ML)/f1-weighted
0,Retweets,concat,0.39 (0.16),0.67 (0.01),0.6 (0.0)
1,Retweets,sum,0.08 (0.12),0.66 (0.01),0.59 (0.01)
2,SO,concat,1.75 (0.11),nan (nan),nan (nan)
3,SO,sum,1.24 (0.09),nan (nan),nan (nan)
4,synthea_full,concat,0.0 (0.0),0.79 (0.07),0.3 (0.07)
5,synthea_full,sum,0.0 (0.0),0.71 (0.14),0.21 (0.1)


Index(['dataset', 'time_enc', 'Best-Test-CIF/LL-#events',
       'Test-NextType(ML)/auc-ovo-weighted', 'Test-NextType(ML)/f1-weighted'],
      dtype='object')

index  \
time_enc                                                                                               concat   
dataset      Best-Test-CIF/LL-#events Test-NextType(ML)/auc-ovo-weighted Test-NextType(ML)/f1-weighted          
Retweets     0.08 (0.12)              0.66 (0.01)                        0.59 (0.01)                      NaN   
             0.39 (0.16)              0.67 (0.01)                        0.6 (0.0)                        0.0   
SO           1.24 (0.09)              nan (nan)                          nan (nan)                        NaN   
             1.75 (0.11)              nan (nan)                          nan (nan)                        2.0   
synthea_full 0.0 (0.0)                0.71 (0.14)                        0.21 (0.1)                       NaN   
                                      0.79 (0.07)                        0.3 (0.07)                       4.0   

                                                                                                             
time_enc                                                                                                sum  
dataset      Best-Test-CIF/LL-#events Test-NextType(ML)/auc-ovo-weighted Test-NextType(ML)/f1-weighted       
Retweets     0.08 (0.12)              0.66 (0.01)                        0.59 (0.01)                    1.0  
             0.39 (0.16)              0.67 (0.01)                        0.6 (0.0)                      NaN  
SO           1.24 (0.09)              nan (nan)                          nan (nan)                      3.0  
             1.75 (0.11)              nan (nan)                          nan (nan)                      NaN  
synthea_full 0.0 (0.0)                0.71 (0.14)                        0.21 (0.1)                     5.0  
                                      0.79 (0.07)                        0.3 (0.07)                     NaN

# Pred

In [5]:
df_filt = dl_runs(runs, selected_tag='V1')

In [6]:
i_groupby_row =['dataset','setting','test_center']
i_groupby_col = ['event_enc','mod']
i_groupby_col=['OUTPUT','INPUT']
i_agg = ['split']
i_metric = ['Best-Test-pred_label/f1-binary','Best-Test-pred_label/AUPRC','Best-Test-pred_label/AUROC']
# i_metric = ['Test-CIF/NLL-#events']
i_stats=['mean','std']
# i_stats=['mean']

#df_filt.iloc[0].config.keys()

d_name = [x for x in df_filt.name]

df_config = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.config]    )

df_summary = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.summary]    )

df = pd.concat([df_config, df_summary],axis=1)
# df.name = df_filt.name
df = df[i_groupby_row + i_groupby_col + i_agg + i_metric]
df




a0 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].size().rename('runs')

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(i_stats).round(3)

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(lambda x:str(round(x.mean(),2))  + ' (' + str(round(x.std(),2))  +')'  ) #  

# a4 = df.groupby(i_by)[i_metric].apply(lambda x:x.round(3).tolist())   # x.round(3).tolist()
a1

a1.reset_index().pivot(index=i_groupby_row,columns=i_groupby_col)



a1.to_excel("aaa.xlsx")

,dataset,setting,test_center,OUTPUT,INPUT,split,Best-Test-pred_label/f1-binary,Best-Test-pred_label/AUPRC,Best-Test-pred_label/AUROC
0,/scratch/hokarami/data_tedam/p19/,mc1,0,none-label,TEDAM,0,0.900398,0.982951,0.998390
1,/scratch/hokarami/data_tedam/p19/,mc1,0,none-label,DAM,0,0.721368,0.892947,0.982703
2,/scratch/hokarami/data_tedam/p19/,mc1,0,none-label,DAM,2,0.680585,0.867488,0.975531
3,/scratch/hokarami/data_tedam/p19/,mc1,0,none-label,TEDAM,1,0.876106,0.979963,0.998207
4,/scratch/hokarami/data_tedam/p19/,mc1,0,none-label,DAM,1,0.640777,0.860533,0.974170
...,...,...,...,...,...,...,...,...,...
102,/scratch/hokarami/data_tedam/p12/,mc1,0,none-label,DAM,2,0.455399,0.526243,0.858446
103,/scratch/hokarami/data_tedam/p12/,mc1,0,none-label,TEDAM,1,0.569733,0.560515,0.920641
104,/scratch/hokarami/data_tedam/p12/,mc1,0,none-label,DAM,1,0.478632,0.488530,0.865652
105,/scratch/hokarami/data_tedam/p12/,mc1,0,none-label,TEDAM,0,0.569014,0.699770,0.945812


Best-Test-pred_label/f1-binary  \
dataset                           setting test_center OUTPUT     INPUT                                  
/scratch/hokarami/data_tedam/p12/ mc1     0           none-label DAM                      0.47 (0.01)   
                                                                 TEDAM                     0.57 (0.0)   
                                          1           none-label DAM                      0.48 (0.03)   
                                                                 TEDAM                    0.59 (0.04)   
                                          2           none-label DAM                      0.51 (0.03)   
                                                                 TEDAM                    0.63 (0.02)   
                                  mc2     0           none-label DAM                       0.43 (nan)   
                                                                 TEDAM                     0.41 (nan)   
                                          1           none-label DAM                       0.43 (nan)   
                                                                 TEDAM                      0.4 (nan)   
                                          2           none-label DAM                       0.42 (nan)   
                                                                 TEDAM                     0.42 (nan)   
                                  sc      0           none-label DAM                      0.39 (0.04)   
                                                                 TEDAM                     0.4 (0.03)   
                                          1           none-label DAM                       0.4 (0.03)   
                                                                 TEDAM                    0.39 (0.02)   
                                          2           none-label DAM                      0.38 (0.02)   
                                                                 TEDAM                     0.4 (0.02)   
/scratch/hokarami/data_tedam/p19/ mc1     0           none-label DAM                      0.68 (0.04)   
                                                                 TEDAM                    0.85 (0.03)   
                                          1           none-label DAM                      0.65 (0.05)   
                                                                 TEDAM                     0.86 (nan)   
                                  mc2     0           none-label DAM                       0.35 (nan)   
                                                                 TEDAM                     0.46 (nan)   
                                          1           none-label DAM                        0.6 (nan)   
                                                                 TEDAM                     0.56 (nan)   
                                  sc      0           none-label DAM                      0.65 (0.01)   
                                                                 TEDAM                    0.63 (0.02)   
                                          1           none-label DAM                      0.49 (0.03)   
                                                                 TEDAM                    0.58 (0.04)   

                                                                       Best-Test-pred_label/AUPRC  \
dataset                           setting test_center OUTPUT     INPUT                              
/scratch/hokarami/data_tedam/p12/ mc1     0           none-label DAM                  0.49 (0.03)   
                                                                 TEDAM                0.63 (0.06)   
                                          1           none-label DAM                   0.5 (0.03)   
                                                                 TEDAM                0.63 (0.04)   
                                          2           none-label DAM                  0.53 (0.06)   
   

Best-Test-pred_label/f1-binary  \
OUTPUT                                                                    none-label   
INPUT                                                                            DAM   
dataset                           setting test_center                                  
/scratch/hokarami/data_tedam/p12/ mc1     0                              0.47 (0.01)   
                                          1                              0.48 (0.03)   
                                          2                              0.51 (0.03)   
                                  mc2     0                               0.43 (nan)   
                                          1                               0.43 (nan)   
                                          2                               0.42 (nan)   
                                  sc      0                              0.39 (0.04)   
                                          1                               0.4 (0.03)   
                                          2                              0.38 (0.02)   
/scratch/hokarami/data_tedam/p19/ mc1     0                              0.68 (0.04)   
                                          1                              0.65 (0.05)   
                                  mc2     0                               0.35 (nan)   
                                          1                                0.6 (nan)   
                                  sc      0                              0.65 (0.01)   
                                          1                              0.49 (0.03)   

                                                                    \
OUTPUT                                                               
INPUT                                                        TEDAM   
dataset                           setting test_center                
/scratch/hokarami/data_tedam/p12/ mc1     0             0.57 (0.0)   
                                          1            0.59 (0.04)   
                                          2            0.63 (0.02)   
                                  mc2     0             0.41 (nan)   
                                          1              0.4 (nan)   
                                          2             0.42 (nan)   
                                  sc      0             0.4 (0.03)   
                                          1            0.39 (0.02)   
                                          2             0.4 (0.02)   
/scratch/hokarami/data_tedam/p19/ mc1     0            0.85 (0.03)   
                                          1             0.86 (nan)   
                                  mc2     0             0.46 (nan)   
                                          1             0.56 (nan)   
                                  sc      0            0.63 (0.02)   
                                          1            0.58 (0.04)   

                                                      Best-Test-pred_label/AUPRC  \
OUTPUT                                                                none-label   
INPUT                                                                        DAM   
dataset                           setting test_center                              
/scratch/hokarami/data_tedam/p12/ mc1     0                          0.49 (0.03)   
                                          1                           0.5 (0.03)   
                                          2                          0.53 (0.06)   
                                  mc2     0                           0.42 (nan)   
                                          1                           0.41 (nan)   
                                          2                           0.41 (nan)   
                                  sc      0                          0.38 (0.02)   
                                          1                          0.35 (0.04)   
                                          

ModuleNotFoundError: No module named 'openpyxl'

In [45]:
df

,dataset,setting,test_center,OUTPUT,INPUT,split,Best-Test-pred_label/f1-binary,Best-Test-pred_label/AUPRC,Best-Test-pred_label/AUROC
0,/scratch/hokarami/data_tedam/p12/,mc2,2,none-label,TEDAM,{0},0.422311,0.378606,0.776002
1,/scratch/hokarami/data_tedam/p12/,mc2,2,none-label,DAM,{0},0.421284,0.407693,0.784718
2,/scratch/hokarami/data_tedam/p12/,mc2,1,none-label,TEDAM,{0},0.398410,0.357587,0.767543
3,/scratch/hokarami/data_tedam/p12/,mc2,1,none-label,DAM,{0},0.430396,0.407364,0.789033
4,/scratch/hokarami/data_tedam/p12/,mc2,0,none-label,TEDAM,{0},0.412155,0.404788,0.789505
...,...,...,...,...,...,...,...,...,...
169,/scratch/hokarami/data_tedam/p12/,mc1,0,single-mark,TE,1,NaN,NaN,NaN
170,/scratch/hokarami/data_tedam/p12/,mc1,0,none-label,TEDAM,0,0.569014,0.699770,0.945812
171,/scratch/hokarami/data_tedam/p12/,mc1,0,none-label,DAM,0,0.459658,0.479535,0.861373
172,/scratch/hokarami/data_tedam/p12/,mc1,0,single-mark,TEDAM,0,NaN,NaN,NaN


# CIF

In [7]:
df_filt = dl_runs(runs, selected_tag='CIF')

df_filt.iloc[0].config.keys()

dict_keys(['w', 'lr', 'log', 'mod', 'per', 'cuda', 'data', 'date', 'demo', 'prof', 'epoch', 'split', 'state', 'w_pos', 'wandb', 'ES_pat', 'device', 'freeze', 'run_id', 'smooth', 'te_d_k', 'te_d_v', 'w_time', 'dataset', 'int_dec', 'setting', 'w_class', 'w_event', 'run_name', 'run_path', 'te_d_rnn', 'time_enc', 'TE_config', 'event_enc', 'next_mark', 'num_demos', 'num_marks', 'num_types', 'pos_alpha', 'te_d_mark', 'te_d_time', 'te_n_head', 'type_loss', 'CIF_config', 'DAM_config', 'batch_size', 'data_label', 'num_states', 'pos_weight', 'str_config', 'te_d_inner', 'te_dropout', 'testloader', 'demo_config', 'mark_detach', 'te_n_layers', 'test_center', 'trainloader', 'user_prefix', 'validloader', 'w_pos_label', 'label_config', 'sample_label', 'weight_decay', 'hparams2write', 'balanced_batch', 'label_loss_fun', 'pred_loss_func', 'w_sample_label', 'next_time_config', 'next_type_config', 'transfer_learning'])

In [43]:
i_groupby_row =['dataset','setting',]
i_groupby_col = ['state']
i_agg = ['test_center','split']
# i_metric = ['Best-Test-pred_label/f1-binary','Best-Test-pred_label/AUPRC']
i_metric = ['Test-CIF/NLL-#events']


#df_filt.iloc[0].config.keys()

d_name = [x for x in df_filt.name]

df_config = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.config]    )

df_summary = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.summary]    )

df = pd.concat([df_config, df_summary],axis=1)#.dropna(axis=1,how='all')
# df.name = df_filt.name
df = df[i_groupby_row + i_groupby_col + i_agg + i_metric]
df





a0 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].size().rename('runs')

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(['mean','std']).round(3)

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(lambda x:str(round(x.mean(),3))  + ' (' + str(round(x.std(),3))  +')' )

# a4 = df.groupby(i_by)[i_metric].apply(lambda x:x.round(3).tolist())   # x.round(3).tolist()
a1

a1.reset_index().pivot(index=i_groupby_row,columns=i_groupby_col)



# a1.to_csv("aaa.csv")

KeyError: "['Test-CIF/NLL-#events'] not in index"